<a href="https://colab.research.google.com/github/mithunkumarsr/NLPNov21/blob/main/Hidden_Markov_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Hidden Markov Model**

In [ ]:
# pretty printing our  distributions
import pandas as pd
from tabulate import tabulate

In [ ]:
def generate_sequence(states,sequence_length):    
    all_sequences = []
    nodes = []    
    depth = sequence_length    
    def gen_seq_recur(states,nodes,depth):
        if depth == 0:
            #print nodes
            all_sequences.append(nodes)
        else:
            for state in states:
                temp_nodes = list(nodes)
                temp_nodes.append(state)
                gen_seq_recur(states,temp_nodes,depth-1)
    
    gen_seq_recur(states,[],depth)
                
    return all_sequences

def score_sequences(sequences,initial_probs,transition_probs,emission_probs,obs):
    
    best_score = -1
    best_sequence = None
    
    sequence_scores = []
    for seq in sequences:
        total_score = 1
        total_score_breakdown = []
        first = True
        for i in range(len(seq)):
            state_score = 1
            # compute transitition probability score
            if first == True:
                state_score *= initial_probs[seq[i]]
                # reset first flag
                first = False
            else:  
                state_score *= transition_probs[seq[i] + "|" + seq[i-1]]
            # add to emission probability score
            state_score *= emission_probs[obs[i] + "|" + seq[i]]
            # update the total score
            #print state_score
            total_score *= state_score
            total_score_breakdown.append(total_score)

        if(total_score > best_score):
            best_score = total_score
            best_sequence = total_score_breakdown

        sequence_scores.append(total_score)
        
    return best_sequence,sequence_scores

def pretty_print_probs(distribs):
    
    rows = set()
    cols = set()
    for val in distribs.keys():
        temp = val.split("|")
        rows.add(temp[0])
        cols.add(temp[1])
        
    rows = list(rows)
    cols = list(cols)
    df = []
    for i in range(len(rows)):
        temp = []
        for j in range(len(cols)):
            temp.append(distribs[rows[i]+"|"+cols[j]])
            
        df.append(temp)
        
    I = pd.Index(rows, name="rows")
    C = pd.Index(cols, name="cols")
    df = pd.DataFrame(data=df,index=I, columns=C)
    
    print(tabulate(df, headers='keys', tablefmt='psql'))

def initializeSequences(states,prob_dict,initial_probs,_obs):
    # Generate list of sequences
    transition_probs = prob_dict[0]
    emission_probs = prob_dict[1]
    seqLen = len(_obs)
    seqs = generate_sequence(states,seqLen)
    # Score sequences
    best_seq,seq_scores = score_sequences(seqs,initial_probs,transition_probs,emission_probs,_obs)
    
    return (seqLen,seqs,best_seq,seq_scores)

#**Viterbi Algorithm**

In [ ]:
import numpy as np

def viterbi(states,prob_dict,initial_probs,_obs):
  '''
  Parameters:
  1. states - list of hidden states ['Hot','Cold','Mild']
  2. prob_dict - tuple that contains the dictionaries transition_probs and emission probs
     example of transition_probs:
     transition_probs = {'Hot|Hot':0.6,'Hot|Mild':0.3,'Hot|Cold':0.1,
                         'Mild|Hot':0.4,'Mild|Mild':0.3,'Mild|Cold':0.2,
                         'Cold|Hot':0.1,'Cold|Mild':0.4,'Cold|Cold':0.5}

     example of emission_probs:
     emission_probs =   {'CasualWear|Hot':0.8,'CasualWear|Mild':0.19,'CasualWear|Cold':0.01,
                         'SemiCasualWear|Hot':0.5,'SemiCasualWear|Mild':0.4,'SemiCasualWear|Cold':0.1,
                         'ApparelWear|Hot':0.01,'ApparelWear|Mild':0.2,'ApparelWear|Cold':0.79}

  3. _obs - list of a sequence of observable states
     example of _obs = ['ApparelWear','CasualWear', 'CasualWear', 'SemiCasualWear']

  Returns:
  1. cache - required for backtracking to achieve the best hidden state sequence
  2. viterbi_list - the index of the max value of this list is required to initiate backward pass of the Viterbi Algorithm
  '''
  # Generate list of sequences
  n = len(states)
  transition_probs = prob_dict[0]
  emission_probs = prob_dict[1]
  viterbi_list = []
  cache = {}

  for state in states:
    viterbi_list.append(initial_probs[state]*emission_probs[_obs[0]+"|"+state])

  for i,ob in enumerate(_obs):
    if i==0: continue
    temp_list = [None]*n
    for j,state in enumerate(states):
      x = -1
      for k,prob in enumerate(viterbi_list):
        val = prob*transition_probs[state+"|"+states[k]]*emission_probs[ob+"|"+state]
        if (x<val):
          x = val
          cache[str(i)+"-"+state] = [states[k],val]
      temp_list[j]= x
    viterbi_list = [x for x in temp_list]
    

  return cache,viterbi_list

def viterbi_backward(states,cache,viterbi_list):
  '''
  Parameters:

  To be used by passing (states , return values of Viterbi Algorithm) as parameters

  1. cache - dictionary that stores state information of algorithm
     example of cache:
     {'1-Hot': ['Cold', 0.015800000000000005], '1-Mild': ['Cold', 0.02528000000000001], '1-Cold': ['Cold', 0.015800000000000005], '2-Hot': ['Hot', 0.007584000000000002], '2-Mild': ['Mild', 0.0014409600000000007], '2-Cold': ['Mild', 0.00010112000000000005]}

  2. viterbi_list - list of numeric values (one corresponding to each state)

  Returns:
  1. best_sequence - list of predicted hidden states 
     example of best_sequence:
     best_sequence = ['Hot','Cold','Cold']...
  
  2. best_sequence_breakdown - list of probabilities at each stage (used for debugging)
     example of best_sequence_breakdown:
     best_sequence_breakdown = [0.5832000000000002, 0.4199040000000001, 0.3023308800000001]
  '''
  num_states = len(states)
  n = len(cache)//num_states
  best_sequence = []
  best_sequence_breakdown=[]
  x = states[np.argmax(np.asarray(viterbi_list))]
  best_sequence.append(x)

  for i in range(n,0,-1):
    val = cache[str(i)+'-'+x][1]
    x = cache[str(i)+'-'+x][0]
    best_sequence = [x] + best_sequence
    best_sequence_breakdown = [val]+best_sequence_breakdown
  
  return best_sequence,best_sequence_breakdown


#**Example 1** (Weather and dressing)

Before actually start working on the dataset, let us understand the HMM using 2 examples.


![alt text](https://drive.google.com/uc?id=1Pkx_Smx3RNkIqtGJZvwBw8_Kz1D9U3oJ)

**Hidden Markov Model** uses different probability theorems for finding a sequence of hidden states for a given observations (visible states). 

So let us assume, that we have lost the data of weather on 10th,11th and 12th of March 2019. Now we need to find the sequence of the weathers on that day (i.e. whether the sequence were *Hot->Hot->Cold* or *Mild->Cold->Hot* or *Cold->Cold->Hot* or some other sequence). 

Note that there are 27 such states (3^3).

Now if you would have had the data then you would have known the sequence of the above *HIDDEN STATES* (hidden because they are unknown) but we have some other sequence (*SEQUENCE OF THE VISIBLE STATES*) with the help of which we can predict the sequence of the *HIDDEN STATES*. 

So you know that on 10th,11th and 12th of March of 2019, an XYZ person had worn a sequence of *Casual Wear->Casual Wear->Semi Casual Wear*. Now this sequence is your observation. Hence observations are generated over visible states. Therefore *'Casual Wear', 'WInter Apparel', 'Semi Casual Wear'* are visible states.

We have 
1. Transition table: 
> That tells us, suppose today is a *Hot* day then what is the probability of tomorrow being a *Mild* day or *Cold* day or a *Hot* day again and so on. Hence `Transition[i][j] = P(today being [state i] | previous day is [state j])`

2. Emission table:
> This tells us the chance of it being *Hot* or *Cold* or *Mild* when XYZ wears a particular wear (*Casual* or *Winter* or *Semi Casual*). Hence `Emission[i][j] = P(wearing [i wear] | it is [j weather])`

Using this information Hidden Markov Model can calculate the weather sequence with the highest chances. 

*To understand in **simple language**, if you are wearing a casual dress, a layman would guess that its a hot day and if you are wearing a woollen dress, a layman would guess that it may be a cold day. This is what HMM helps us in calculating. It observes a visible sequence to calculate the chances of a hidden sequence.*

To understand the **MATH** behind it refer : https://web.stanford.edu/~jurafsky/slp3/A.pdf



In [ ]:
def get_probs_ex1():
  # transition probabilities
  transition_probs = {'H|H':0.6,'H|M':0.3,'H|C':0.1,
                      'M|H':0.4,'M|M':0.3,'M|C':0.2,
                      'C|H':0.1,'C|M':0.4,'C|C':0.5}

  # emission probabilities (CW-Casual Wear , SW-Semi Casual Wear , AW - Apparel)
  emission_probs =   {'CW|H':0.8,'CW|M':0.19,'CW|C':0.01,
                      'SW|H':0.5,'SW|M':0.4,'SW|C':0.1,
                      'AW|H':0.01,'AW|M':0.2,'AW|C':0.79}
                      
  return transition_probs,emission_probs

#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@

# hidden states
states1 = ['H','M','C']

# set of observations
obs = ['AW','SW','CW'] 

# initial state probability distribution (our priors)
initial_probs1 = {'H':0.3,'M':0.3,'C':0.4}

# Generate list of sequences
sequence_length,sequences,best_sequence,sequence_scores = initializeSequences(states1,get_probs_ex1(),initial_probs1,obs)

print("Initial Distributions")
print(initial_probs1)

transition_probs, emission_probs = get_probs_ex1()

print("\nTransition Probabilities")
pretty_print_probs(transition_probs)

print("\nEmission Probabilities")
pretty_print_probs(emission_probs)

print("\nScores")
# Display sequence scores
for i in range(len(sequences)):
    print("Sequence:%10s,Score:%0.4f" % (sequences[i],sequence_scores[i]))

# The best sequence for the given observation (obs) and initial chances of the hidden states (initial_probs)
best_seq_n = np.argmax(sequence_scores)
print("\nBest Sequence:",sequences[best_seq_n],best_sequence)
print()
cache,l = viterbi(states1,get_probs_ex1(),initial_probs1,obs)

print("Path generated by viterbi algorithm by reducing computation...",cache,sep="\n")
print()
print("Finding the best hidden sequence from the path generated by viterbi algorithm...")
print(viterbi_backward(states1,cache,l),np.max(np.asarray(l)))

Initial Distributions
{'H': 0.3, 'M': 0.3, 'C': 0.4}

Transition Probabilities
+--------+-----+-----+-----+
| rows   |   H |   C |   M |
|--------+-----+-----+-----|
| H      | 0.6 | 0.1 | 0.3 |
| C      | 0.1 | 0.5 | 0.4 |
| M      | 0.4 | 0.2 | 0.3 |
+--------+-----+-----+-----+

Emission Probabilities
+--------+------+------+------+
| rows   |    H |    C |    M |
|--------+------+------+------|
| CW     | 0.8  | 0.01 | 0.19 |
| AW     | 0.01 | 0.79 | 0.2  |
| SW     | 0.5  | 0.1  | 0.4  |
+--------+------+------+------+

Scores
Sequence:['H', 'H', 'H'],Score:0.0004
Sequence:['H', 'H', 'M'],Score:0.0001
Sequence:['H', 'H', 'C'],Score:0.0000
Sequence:['H', 'M', 'H'],Score:0.0001
Sequence:['H', 'M', 'M'],Score:0.0000
Sequence:['H', 'M', 'C'],Score:0.0000
Sequence:['H', 'C', 'H'],Score:0.0000
Sequence:['H', 'C', 'M'],Score:0.0000
Sequence:['H', 'C', 'C'],Score:0.0000
Sequence:['M', 'H', 'H'],Score:0.0043
Sequence:['M', 'H', 'M'],Score:0.0007
Sequence:['M', 'H', 'C'],Score:0.0000
Sequen

In the above example, initial probabilities of it being a hot day, cold day or a mild day were equal. But since the observable sequence (`obs`) was `['CW','CW','CW']`, therefore our HMM predicted that the three days would have been `['H', 'H', 'H']`

##**Example 2** (Simple Part of speech tagging)

In [ ]:
def get_probs_ex2():
  transition_probs = {'Noun|Noun':0.1,'Noun|Verb':0.1,'Noun|Determiner':0.8,
                      'Verb|Noun':0.8,'Verb|Verb':0.1,'Verb|Determiner':0.1,
                      'Determiner|Noun':0.1,'Determiner|Verb':0.8,'Determiner|Determiner':0.1}
  emission_probs = {'Bob|Noun':0.9,'ate|Noun':0.05,'the|Noun':0.05,'fruit|Noun':0.9,\
                    'Bob|Verb':0.05,'ate|Verb':0.9,'the|Verb':0.05,'fruit|Verb':0.05,\
                    'Bob|Determiner':0.05,'ate|Determiner':0.05,'the|Determiner':0.9,'fruit|Determiner':0.05}
  return transition_probs,emission_probs

#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@#@

# generate new sequences
states2 = ['Noun','Verb','Determiner']

initial_probs2 = {'Noun':0.9,'Verb':0.05,'Determiner':0.05}

transition_probs, emission_probs = get_probs_ex2()

print("Initial Distributions")
print(initial_probs2)
print("\nTransition Probabilities")
pretty_print_probs(transition_probs)
print("\nEmission Probabilities")
pretty_print_probs(emission_probs)

Initial Distributions
{'Noun': 0.9, 'Verb': 0.05, 'Determiner': 0.05}

Transition Probabilities
+------------+--------+--------+--------------+
| rows       |   Verb |   Noun |   Determiner |
|------------+--------+--------+--------------|
| Verb       |    0.1 |    0.8 |          0.1 |
| Noun       |    0.1 |    0.1 |          0.8 |
| Determiner |    0.8 |    0.1 |          0.1 |
+------------+--------+--------+--------------+

Emission Probabilities
+--------+--------+--------+--------------+
| rows   |   Verb |   Noun |   Determiner |
|--------+--------+--------+--------------|
| ate    |   0.9  |   0.05 |         0.05 |
| fruit  |   0.05 |   0.9  |         0.05 |
| Bob    |   0.05 |   0.9  |         0.05 |
| the    |   0.05 |   0.05 |         0.9  |
+--------+--------+--------+--------------+


In [ ]:
obs = ['Bob','ate','the','fruit']

# print results
print("\nScores")

# Generate list of sequences
sequence_length,sequences,best_sequence,sequence_scores = initializeSequences(states2,get_probs_ex2(),initial_probs2,obs)

# Display sequence scores
# Total number of sequences are 3^4 = 81
for i in range(len(sequences)):
    print("Sequence:%-60s Score:%0.6f" % (sequences[i],sequence_scores[i]))
    
# Display the winning score
best_seq_n = np.argmax(sequence_scores)
print("\nBest Sequence:",sequences[best_seq_n],best_sequence)
print()
cache,l = viterbi(states2,get_probs_ex2(),initial_probs2,obs)
print("Path generated by viterbi algorithm by reducing computation...",cache,sep="\n")
print()
print("Finding the best hidden sequence from the path generated by viterbi algorithm...")
print(viterbi_backward(states2,cache,l),np.max(np.asarray(l)))


Scores
Sequence:['Noun', 'Noun', 'Noun', 'Noun']                             Score:0.000002
Sequence:['Noun', 'Noun', 'Noun', 'Verb']                             Score:0.000001
Sequence:['Noun', 'Noun', 'Noun', 'Determiner']                       Score:0.000000
Sequence:['Noun', 'Noun', 'Verb', 'Noun']                             Score:0.000015
Sequence:['Noun', 'Noun', 'Verb', 'Verb']                             Score:0.000001
Sequence:['Noun', 'Noun', 'Verb', 'Determiner']                       Score:0.000006
Sequence:['Noun', 'Noun', 'Determiner', 'Noun']                       Score:0.000262
Sequence:['Noun', 'Noun', 'Determiner', 'Verb']                       Score:0.000002
Sequence:['Noun', 'Noun', 'Determiner', 'Determiner']                 Score:0.000002
Sequence:['Noun', 'Verb', 'Noun', 'Noun']                             Score:0.000262
Sequence:['Noun', 'Verb', 'Noun', 'Verb']                             Score:0.000117
Sequence:['Noun', 'Verb', 'Noun', 'Determiner']          

We have correctly tagged the part of speech in 



1. Bob - Noun

2. ate - Verb

3. the - Determiner

4. fruit - Noun



#**Data for part of speech tagging**

In [ ]:
import numpy as np
import nltk
nltk.download('treebank')
nltk.download('universal_tagset')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data...
[nltk_data]   Unzipping taggers/universal_tagset.zip.


True

In [ ]:
from nltk.corpus import treebank
sentences = treebank.tagged_sents(tagset='universal')

![alt text](https://drive.google.com/uc?id=1fJQAmAvb9-17J6gcdvUJHJGyy6Y-hctd)

In [ ]:
len(sentences)

3914

In [ ]:
tagged_sentence = nltk.corpus.treebank.tagged_sents(tagset='universal')
print("Number of Tagged Sentences ",len(tagged_sentence))
tagged_words=[tup for sent in tagged_sentence for tup in sent]
print("Total Number of Tagged words", len(tagged_words))
vocab=set([word for word,tag in tagged_words])
print("Vocabulary of the Corpus",len(vocab))
tags=set([tag for word,tag in tagged_words])
print("Number of Tags in the Corpus ",len(tags))

Number of Tagged Sentences  3914
Total Number of Tagged words 100676
Vocabulary of the Corpus 12408
Number of Tags in the Corpus  12


In [ ]:
print(tags)

{'NOUN', 'DET', 'ADV', 'PRON', 'PRT', 'ADP', 'NUM', 'CONJ', 'VERB', 'X', '.', 'ADJ'}


#**Generating Dictionary for our hidden states (Part of speech tags)** -  `transition_probs`

`transition_probs` is a dictionary that tells the probability of getting a particular tag as the next POS tag given that which tag had occured previously. 

Hence using `transition_probs` we get:
> `P['next state'|'previous state']`



In [ ]:
import numpy as np
states = list(tags)
print(states)

['NOUN', 'DET', 'ADV', 'PRON', 'PRT', 'ADP', 'NUM', 'CONJ', 'VERB', 'X', '.', 'ADJ']


In [ ]:
# Generating the martix for calculating transition_probs for the dataset

tr_matrix = np.zeros((len(states),len(states)))

for sentence in tagged_sentence:
  for i in range(len(sentence)):
    if i==0: continue
    tr_matrix[states.index(sentence[i-1][1])][states.index(sentence[i][1])]+=1

print(tr_matrix.astype(int))

[[7623  377  490  135 1267 5100  273 1229 4240  837 6926  350]
 [5569   48  110   31    2   81  193    4  346  398  154 1788]
 [ 100  218  252   48   45  376  100   22 1093   73  432  411]
 [ 573   26   93   21   34   62   20   14 1329  254  111  200]
 [ 796  326   32   58    6   66  183    7 1291   43  138  273]
 [3173 3194  133  679   14  167  618    8   82  342  392 1050]
 [1252   11   10    5   96  124  656   48   64  746  414  118]
 [ 792  270  124  133   11  120   94    1  355   19   80  266]
 [1497 1822 1110  482  426 1239  310   73 2292 2954  475  884]
 [ 410  361  170  367 1221  955   18   68 1354  494 1082  112]
 [1467 1114  412  486   23  565  915  482 1003  229  776  359]
 [4474   31   30    4   69  497  133  108   77  134  415  425]]


In [ ]:
tr_matrix /= np.sum(tr_matrix,keepdims=True,axis=1)
print(tr_matrix)

[[2.64256248e-01 1.30689500e-02 1.69861684e-02 4.67986272e-03
  4.39213783e-02 1.76794814e-01 9.46372240e-03 4.26040836e-02
  1.46982355e-01 2.90151489e-02 2.40094291e-01 1.21329774e-02]
 [6.38353966e-01 5.50206327e-03 1.26088950e-02 3.55341586e-03
  2.29252636e-04 9.28473177e-03 2.21228794e-02 4.58505273e-04
  3.96607061e-02 4.56212746e-02 1.76524530e-02 2.04951857e-01]
 [3.15457413e-02 6.87697161e-02 7.94952681e-02 1.51419558e-02
  1.41955836e-02 1.18611987e-01 3.15457413e-02 6.94006309e-03
  3.44794953e-01 2.30283912e-02 1.36277603e-01 1.29652997e-01]
 [2.09353307e-01 9.49945195e-03 3.39788089e-02 7.67263427e-03
  1.24223602e-02 2.26525393e-02 7.30727073e-03 5.11508951e-03
  4.85568140e-01 9.28023383e-02 4.05553526e-02 7.30727073e-02]
 [2.47281765e-01 1.01273687e-01 9.94097546e-03 1.80180180e-02
  1.86393290e-03 2.05032619e-02 5.68499534e-02 2.17458838e-03
  4.01056229e-01 1.33581858e-02 4.28704567e-02 8.48089469e-02]
 [3.22066585e-01 3.24198132e-01 1.34997970e-02 6.89200162e-02
  1

In [ ]:
#Hence we get our transition matrix. Now we need to create the dictionary transition_probs

def get_transition_probs(states,tr_matrix):
  state_dict = {}
  for i,state in enumerate(states):
    state_dict[i]=state
  transition_probs = {}
  for i in range(tr_matrix.shape[0]):
    for j in range(tr_matrix.shape[1]):
      transition_probs[state_dict[j]+'|'+state_dict[i]] = tr_matrix[i][j]
  return transition_probs

In [ ]:
def get_tr_matrix(states,tagged_sentence):
  tr_matrix = np.zeros((len(states),len(states)))
  for sentence in tagged_sentence:
    for i in range(len(sentence)):
      if i==0: continue
      tr_matrix[states.index(sentence[i-1][1])][states.index(sentence[i][1])]+=1
  tr_matrix /= np.sum(tr_matrix,keepdims=True,axis=1)
  return tr_matrix

#**Generating Dictionary for our observable states (words)** - `emission_probs`

In [ ]:
vocab = sorted(list(vocab))
em_matrix = np.zeros((len(vocab),len(states)))
print(em_matrix.shape)

for sentence in tagged_sentence:
  for word,tag in sentence:
    em_matrix[vocab.index(word)][states.index(tag)]+=1

print(em_matrix.astype(int))

(12408, 12)
[[  0   0   0 ...   0   6   0]
 [  0   0   0 ...   0  16   0]
 [  0   0   0 ...   0 718   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  2   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]


In [ ]:
sum_matrix = np.sum(em_matrix,keepdims=True,axis=0)
em_matrix = np.divide(em_matrix, sum_matrix, out=np.zeros_like(em_matrix), where=sum_matrix!=0)
print(em_matrix)

[[0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  5.12163892e-04 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  1.36577038e-03 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  6.12889458e-02 0.00000000e+00]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.92832646e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


In [ ]:
#Hence we get our emission matrix. Now we need to create the dictionary transition_probs
def get_emission_probs(states,vocab,em_matrix):
  state_dict = {}
  for i,state in enumerate(states):
    state_dict[i]=state
  emission_probs = {}
  for i in range(em_matrix.shape[0]):
    for j in range(em_matrix.shape[1]):
      emission_probs[vocab[i]+'|'+state_dict[j]] = em_matrix[i][j]
  return emission_probs

In [ ]:
def get_em_matrix(states,vocab,tagged_sentence):
  vocab = sorted(list(vocab))
  em_matrix = np.zeros((len(vocab),len(states)))
  for sentence in tagged_sentence:
    for word,tag in sentence:
      em_matrix[vocab.index(word)][states.index(tag)]+=1
  sum_matrix = np.sum(em_matrix,keepdims=True,axis=0)
  em_matrix = np.divide(em_matrix, sum_matrix, out=np.zeros_like(em_matrix), where=sum_matrix!=0)
  return em_matrix

In [ ]:
def get_probs(states,vocab,tagged_sentence):
  tr_matrix = get_tr_matrix(states,tagged_sentence)
  em_matrix = get_em_matrix(states,vocab,tagged_sentence)
  transition_probs = get_transition_probs(states,tr_matrix)
  emission_probs = get_emission_probs(states,vocab,em_matrix)
  return transition_probs,emission_probs

In [ ]:
def get_initial_probs(states,tagged_sentence):
  initial_list = [0]*len(states)
  for sent in tagged_sentence:
    tag = sent[0][1]
    initial_list[states.index(tag)]+=1
  initial_list = np.asarray(initial_list)
  initial_list = initial_list/np.sum(initial_list,keepdims=True)
  initial_probs = {}
  for i,state in enumerate(states):
    initial_probs[state] = initial_list[i]
  return initial_probs

In [ ]:
def get_observation(sent):
  ob = []
  ob_tags = []
  for word,tag in sent:
    ob.append(word)
    ob_tags.append(tag)
  return ob,ob_tags

In [ ]:
initial_probs = get_initial_probs(states,tagged_sentence)

probs = get_probs(states,vocab,tagged_sentence)

# **Testing our Hidden Markov Model on the dataset**

In [ ]:
import random
ob = random.choice(tagged_sentence)
obs,ob_tags = get_observation(ob)

cache,l = viterbi(states,probs,initial_probs,obs)
print("Finding the best hidden sequence from the path generated by viterbi algorithm...")
print()
best_seq,_ = viterbi_backward(states,cache,l)
print("TAGS ACHIEVED USING HIDDEN MARKOV MODEL-",best_seq,sep="\n")
print()
print("ACTUAL TAGS-",ob_tags,sep="\n")

Finding the best hidden sequence from the path generated by viterbi algorithm...

TAGS ACHIEVED USING HIDDEN MARKOV MODEL-
['CONJ', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'VERB', 'X', 'PRON', 'VERB', 'ADJ', 'X', 'PRT', 'VERB', '.', 'ADP', 'ADV', 'ADP', 'NOUN', '.', 'ADV', 'ADP', 'NOUN', 'CONJ', 'NOUN', '.']

ACTUAL TAGS-
['CONJ', 'NOUN', 'ADP', 'NOUN', 'NOUN', 'VERB', 'X', 'PRON', 'VERB', 'ADJ', 'X', 'PRT', 'VERB', '.', 'ADJ', 'ADV', 'ADP', 'NOUN', '.', 'ADV', 'ADP', 'NOUN', 'CONJ', 'NOUN', '.']


#**Further improvements**

If we carefully examine the dataset and our code, we would realise that we have actually overfitted the data. Hence we are getting extremely accurate results without any preprocessing. For example, our `vocab` that we generate from the dataset has a huge count of irrelevant items. The `vocab` explicitly includes numbers and patterns like ' 12 ' , ' 3762 ' , ' \* U \* ' , ' \* v - 10 ' and even proper nouns like ' David ' , ' Sam ' etc. We must pre process our dataset so that our model can generalise even better. Using Hidden Markov Model demands for certain preprocessing of the data that has not been considered in the above notebook, and hence can be worked upon for further improvements. 

#**Conclusion**


1. Hidden Markov Model has been built using Viterbi Algorithm 

2. Part of speech tagging has been implemented using HMM model.

3. We have used the `tree-bank` dataset from `nltk.corpus` to test our model.